# Introduction

This notebook is dedicated to the data processing of the laboratory rhizotron experiment. 
Two types of data are collected, an ERT dataset and a MALM. A classical ERT inversion is done while no inversion is required for the MALM as the potential value are analysed using a simple descriptive approach wich consist in plotting the gradient of the potential field * the conductivity of the medium.
This operation led to the identification of the direction and intensity of the current flux without having to invert the MALM data.

# Processing

In [1]:
import os
import matplotlib.pyplot as plt

#!pip install resipy
#import resipy

#!conda install -f pybert
#!conda install -c gimli pybert
import pybert as pb
from pybert import tdip


#import pygimli
import numpy as np

# My own library
from rhizo.utils_rhizo import fct_utils as FU
from rhizo.run_ERT_invert_plant import invert_Resipy_ERT, invert_pygimli_ERT

### Import all the necessary packages
pybert is required in order to: 
- invert ERT data
- import TDIP data
- filter TDIP data

Define survey date (change date accordingly)

In [2]:
date = '0209' #  '1712' 0112 '1310' '1611' # (ddmm)

In [3]:
main = os.getcwd()
os.chdir(main)
geomPath, meshPath, icsdPath, figpath = FU.definePath(main,date)
print(figpath)

./fig/0209/


### Define survey parameters
if invERT is set to FALSE, then waterRes needs to be defined according to the measured conductivity measured with the probe (in microS/cm)

(see excel file for correspondance dates/files/parameters)

In [4]:
invERT = True
waterRes = (1/855)*1e4 #21.23 # water resistivity (Ohm.m) converted from microS/cm 767, 477 
rec = True # reciprocal analysis
A = 72-1 # Id electrode A
B = 65-1 # Id electrode B
injection_duration = 2 # time of injection

Load ERT and MALM files 

(nothing to change providing all the raw file data respect the naming convention ERT_mmdd.bin (mm=month, dd=day))

In [5]:
# filename located in the raw data folder
# MALMIP_0122 MALMIP_0113 'MALMIP1217' 'MALMIP1201.bin' 'MALMIP1013' 'MALMIP1116.bin'
inputfileMALM = 'MALMIP_' + date + '.bin' #  
inputfileERT = 'ERT_' + date + '.bin' #

Filter data (no change needed in normal config)

In [6]:
rmvInvalid = False # True if you want to write filtered files/ False for raw data
rmv_outliers = False
rmv_id= None # remove an electrode from is id

split_Nfix = [True, 71-1]
Nfix = 71-1 #71-1  #None 71-1 # Id electrode N , put None if N is varying

Plot TDIP data options (no change needed)

In [7]:
plotCC = False # show Cole-Cole fitted parameters (to compare with literature)
all_gates= True
if not all_gates: 
    gateIP = 3 
else: gateIP = None # id extracted time window in mV/V (?)

### Run ERT inversion

Load geometry and mesh (fwd and inv) - (no change needed)

In [8]:
if invERT:
    RemLineNb, Injection, coordE, pointsE= FU.load_geom(geomPath) # geometry file containing electrodes position including remotes 
    mesh3d_fwd, sensors = FU.mesh_import(meshPath + 'BaseRhizo_Vrte.msh')
    mesh3d_inv, sensors = FU.mesh_import(meshPath + 'BaseRhizo_Vrte_inv.msh')

Reading ./mesh/BaseRhizo_Vrte.msh... 

  Nodes: 8806
  Entries: 46678
    Points: 324
    Lines: 0
    Triangles: 13426
    Quads: 0
    Tetrahedra: 32928 

Creating mesh object... 

  Dimension: 3-D
  Boundary types: 1 (-1,)
  Regions: 1 (2,)
  Marked nodes: 324 (99, 991, 999, 1000)

Done. 

  Mesh: Nodes: 8806 Cells: 32928 Boundaries: 72569
-999
-1000
Reading ./mesh/BaseRhizo_Vrte_inv.msh... 

  Nodes: 3462
  Entries: 17622
    Points: 72
    Lines: 0
    Triangles: 5998
    Quads: 0
    Tetrahedra: 11552 

Creating mesh object... 

  Dimension: 3-D
  Boundary types: 1 (-1,)
  Regions: 1 (2,)
  Marked nodes: 72 (99,)

Done. 

  Mesh: Nodes: 3462 Cells: 11552 Boundaries: 26103


Run with resipy code

In [9]:
#if invERT:
#        inputfileERTcsv = 'ERT_' + date + '.csv'
#    k = invert_Resipy_ERT(inputfileERTcsv)
#    k.showResults()
#    k.saveInvPlots(figpath + 'figs')

Run with pygimli code

In [ ]:
if invERT:
    model = invert_pygimli_ERT(inputfileERT,sensors,mesh3d_inv,date)
    pg.show(mesh3d_inv,data=model,notebook=True)
    mesh3d_inv.addData('model',model)
    mesh3d_inv.exportVTK('model' + date + '.vtk')
    
    plotter, _ = pg.show(mesh3d_inv, data=model,
                         alpha=0.9, hold=True, notebook=True)
    plotter.view_xy()
    #plotter.clim([20, 60])
    plotter.show()
    plotter.screenshot('model' + date + '.png')

23/02/21 - 10:22:51 - pyGIMLi - INFO - imported:  Data: Electrodes: 72 data: 4968
23/02/21 - 10:22:51 - pyGIMLi - INFO - Cache C:\Users\gcass\anaconda3\envs\pygimli\lib\site-packages\pygimli\physics\ert\ert.py:createGeometricFactors restored (0.0s x 7): C:\Users\gcass\AppData\Roaming\pygimli\Cache\10960051449893461007
23/02/21 - 10:22:52 - pyGIMLi - INFO - Found 1 regions.
23/02/21 - 10:22:52 - pyGIMLi - INFO - Creating forward mesh from region infos.
23/02/21 - 10:22:53 - pyGIMLi - INFO - Creating refined mesh (H2) to solve forward task.
23/02/21 - 10:22:56 - pyGIMLi - INFO - Starting inversion.
23/02/21 - 10:22:56 - pyGIMLi - INFO - Set default startmodel to median(data values)=126.82753380895355
23/02/21 - 10:22:56 - pyGIMLi - INFO - Created startmodel from forward operator: 11552 [126.82753380895355,...,126.82753380895355]


fop: <pygimli.physics.ert.ert.ERTModelling object at 0x0000023AC37D7870>
Data transformation: <pygimli.core._pygimli_.RTransLogLU object at 0x0000023AC37D79B0>
Model transformation: <pygimli.core._pygimli_.RTransLog object at 0x0000023AC37D78B0>
min/max (data): 0.10/5030
min/max (error): 10.00%/10.96%
min/max (start model): 127/127
--------------------------------------------------------------------------------


### MALM processing

In [ ]:
#%% Import data TDIP
IPcurves = tdip.TDIPdata('./raw_data/' + inputfileMALM) # e.g. ABEM or Syscal TXT export
valid = np.ones(len(IPcurves.data('m')))
IPcurves_f = tdip.TDIPdata('./raw_data/' + inputfileMALM)

In [ ]:
if split_Nfix[0]:

    IPcurves.data('m').array()
    idn = np.where(IPcurves.data('m')==split_Nfix[1])[0]
    idm = np.where(IPcurves.data('n')==split_Nfix[1])[0]
    idfix = list(idn)  #+ list(idm)
    
    IPcurves.data('a')[idfix].array()
    IPcurves.data('b')[idfix].array()
    id_elec_2rmv = idfix # remove Nfix electrodes
    
    if Nfix is not None:
        a = set(list(range(0, len(IPcurves.data('m')))))
        id_elec_2rmv = a.difference(set(idfix))
        id_elec_2rmv = list(id_elec_2rmv)
    
    IPcurves_f, valid_split = FU.filterTDIP(IPcurves_f,id_elec_2rmv)

Remove outliers 

In [ ]:
if rmv_outliers: 
    id_outliers = np.where(abs(IPcurves_f.data['M1'])>100)[0]
    id_elec_2rmv= list(id_outliers)
    if rmv_id: 
        id_elec_2rmv.append(rmv_id)
    IPcurves_f, valid_outliers = filterTDIP(IPcurves_f,id_elec_2rmv)

j=0
try:
    valid_split
except NameError:
    valid_split = np.ones(len(IPcurves.data('m')))
    
for i, v in enumerate(valid_split):
    if v==1: # if valid in split
        # check if valid in outliers
        if rmv_outliers:
            if valid_outliers[j] == 0:
                valid[i] = 0
                print(i)
            else:
                valid[i] = 1
            j = j + 1
    else: 
        valid[i] = 0    

Show decay

In [ ]:
IPcurves_f.showDecay(nr=np.arange(0,len(IPcurves_f.data['a'])), showFit=False, 
                   yscale='linear',xscale='linear')

In [ ]:
Obs_raw = pb.importer.importSyscalPro('./raw_data/' + inputfileMALM) 
Obs, dataABMN=  FU.PrepareMALMData('./raw_data/' + inputfileMALM, Rec=False, DevErr=1,
                           MinV=1, MaxRc=1, Kfact=1, MinMaxAppRes=1, 
                           SwE=False, 
                           valid=valid)
coordE_f = []
for i, mi in enumerate(Obs['m']):
    if mi==Nfix:
       mi=Obs['n'][i]
    id_coordE_f = np.where(mi+1==coordE[:,0])[0]
    #if len(id_coordE_f) > 1:
    coordE_f.append(coordE[id_coordE_f[0],:])
coordE_f = np.array(coordE_f)

Quiver plot= gradient(V)*conductivity

Results are saved in figs/date/ folder

In [ ]:
if Nfix is not None:

    FU.streamlines(coordE_f, Obs('r').array(), waterRes,
                   sensors=sensors, A=A, B=B, Nfix=Nfix,
                    vmin=-300, vmax=200)
    plt.savefig(figpath + 'streamlines_PV.png')

    fig, ax = plt.subplots(nrows=1, ncols=4,figsize=(20,5))
    for i, g in enumerate(range(1,20,5)):
        FU.streamlines(coordE_f, Obs('M'+str(g)).array(), waterRes,
                       sensors=sensors, A=A, B=B, Nfix=Nfix, ax=ax[i],
                       vmin=-10, vmax=10)
        ax[i].set_title('Gate t:' + str(IPcurves.t[g-1]) + 's')
        plt.tight_layout()
        plt.savefig(figpath + 'streamlines_transients.png')

    fig, ax = plt.subplots(nrows=1, ncols=4,figsize=(20,5))
    for i, g in enumerate(range(1,20,5)):
        FU.streamlines(coordE_f, Obs('M'+str(g)).array(), waterRes,
                       sensors=sensors, A=A, B=B, Nfix=Nfix, ax=ax[i]
                       )
        ax[i].set_title('Gate t:' + str(IPcurves.t[g-1]) + 's')
        plt.tight_layout()
        plt.savefig(figpath + 'streamlines_transients2.png')

This notebook is dedicated to the data processing of the laboratory rhizotron experiment. 
Two types of data are collected, ERT and MALM. A classical ERT inversion is followed by an extensive analysis of the MALM data. As for the MALM, no inversion is required as the potential value are analysed using a simple descriptive approach wich consist in plotting the gradient of the potential field * the conductivity of the medium. This operation led to the identification of the direction and intensity of the current flux without having to invert the MALM data.